# LSTM Experiments


## Imports

In [7]:
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import TextVectorization, Embedding, Bidirectional, LSTM, Dense

## Read Data

In [2]:
BASE_PATH = r"C:\Users\joehi\Documents\Work\Uni - Year 4\COMP61332\Coursework\dataset\data\json"
# FILE = r"\train.json"
FILE = r"\dev.json"

df = pd.read_json(BASE_PATH + FILE)

In [12]:
df.head(5)

,id,docid,relation,token,subj_start,subj_end,obj_start,obj_end,subj_type,obj_type,stanford_pos,stanford_ner,stanford_head,stanford_deprel
0,e7798fb926b9403cfcd2,APW_ENG_20101103.0539,per:title,"[At, the, same, time, ,, Chief, Financial, Off...",8,9,12,12,PERSON,TITLE,"[IN, DT, JJ, NN, ,, NNP, NNP, NNP, NNP, NNP, M...","[O, O, O, O, O, O, O, O, PERSON, PERSON, O, O,...","[4, 4, 4, 12, 12, 10, 10, 10, 10, 12, 12, 0, 1...","[case, det, amod, nmod, punct, compound, compo..."
1,e779865fb96bbbcc4ca4,APW_ENG_20080229.1401.LDC2009T13,no_relation,"[U.S., District, Court, Judge, Jeffrey, White,...",17,18,4,5,PERSON,PERSON,"[NNP, NNP, NNP, NNP, NNP, NNP, IN, NNP, VBD, D...","[LOCATION, O, O, O, PERSON, PERSON, O, O, O, O...","[6, 6, 6, 6, 6, 9, 8, 6, 0, 11, 9, 13, 11, 20,...","[compound, compound, compound, compound, compo..."
2,e7798ae9c0adbcdc81e7,APW_ENG_20090707.0488,per:city_of_death,"[PARIS, 2009-07-07, 11:07:32, UTC, French, med...",9,9,0,0,PERSON,CITY,"[NNP, CD, CD, NNP, NNP, NNS, RBR, VBD, IN, NNP...","[LOCATION, TIME, TIME, TIME, MISC, O, O, O, O,...","[6, 6, 6, 6, 6, 8, 8, 0, 16, 16, 10, 10, 12, 1...","[compound, nummod, nummod, compound, compound,..."
3,e779865fb9bddb4eccbc,LTW_ENG_20070522.0147.LDC2009T13,no_relation,"[The, current, holdings, of, Blackstone-operat...",13,14,1,1,ORGANIZATION,DATE,"[DT, JJ, NNS, IN, JJ, NNS, VBP, NNP, NNP, ,, N...","[O, DATE, O, O, MISC, O, O, ORGANIZATION, ORGA...","[3, 3, 7, 6, 6, 3, 0, 9, 7, 9, 12, 9, 9, 15, 9...","[det, amod, nsubj, case, amod, nmod, ROOT, com..."
4,e779865fb97262ce4da3,eng-NG-31-108589-8120474,no_relation,[http://groups.yahoo.com/group/aspartameNM/mes...,27,27,53,53,ORGANIZATION,PERSON,"[NN, NN, IN, DT, JJ, JJ, NN, IN, NN, NN, :, NN...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[2, 0, 7, 7, 7, 7, 2, 10, 10, 7, 2, 13, 2, 13,...","[compound, ROOT, case, det, amod, amod, nmod, ..."


In [6]:
d = df.iloc[2]
print(d, "\n\n")

for i in range(len(d["token"])):
    print(d["token"][i].ljust(15), d["stanford_pos"][i], d["stanford_ner"][i].ljust(10), d["stanford_head"][i], d["stanford_deprel"][i], sep="\t")

id                                              e7798ae9c0adbcdc81e7
docid                                          APW_ENG_20090707.0488
relation                                           per:city_of_death
token              [PARIS, 2009-07-07, 11:07:32, UTC, French, med...
subj_start                                                         9
subj_end                                                           9
obj_start                                                          0
obj_end                                                            0
subj_type                                                     PERSON
obj_type                                                        CITY
stanford_pos       [NNP, CD, CD, NNP, NNP, NNS, RBR, VBD, IN, NNP...
stanford_ner       [LOCATION, TIME, TIME, TIME, MISC, O, O, O, O,...
stanford_head      [6, 6, 6, 6, 6, 8, 8, 0, 16, 16, 10, 10, 12, 1...
stanford_deprel    [compound, nummod, nummod, compound, compound,...
Name: 2, dtype: object 


PARIS   

In [14]:
# Define constants
INPUT_SIZE = ...
EMEDDING_SIZE = 300
LSTM_UNITS = 128

VOCAB_SIZE = 20000
MAX_LEN = 100
NUM_LABELS = df["relation"].nunique()

In [40]:
def pad_token_array(input_array, max_len):
    mask = np.arange(max_len) < np.array([len(seq) for seq in input_array])[:, None]
    padded_array = np.full((len(input_array), max_len), '', dtype='object')
    padded_array[mask] = np.concatenate(input_array)
    return padded_array

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import TextVectorization

all_tokens = np.concatenate(df['token'].values)
tokens = pad_token_array(df['token'].values, MAX_LEN)

vectoriser = TextVectorization(max_tokens=VOCAB_SIZE, output_sequence_length=MAX_LEN)
vectoriser.adapt(all_tokens)
sequences = np.array(vectoriser(tokens))

In [62]:
sentences = []
for sentence in df['token'].values:
    sentences.append(" ".join(sentence))

sequences2 = np.array(vectoriser(sentences))
print(sequences2, sequences2.shape, "\n")
sequences = np.array(vectoriser(tokens.reshape(*tokens.shape, 1)))
# print(sequences, sequences.shape, "\n")
a = sequences[:,:,0]
print(a, a.shape)

print(np.bincount((sequences2 == a).flatten())/np.bincount((sequences2 == a).flatten()).sum())

[[   19     2   655 ...     0     0     0]
 [   37  1271   342 ...     0     0     0]
 [  303     1     1 ...     0     0     0]
 ...
 [  108    60  1113 ...     0     0     0]
 [  448   501    15 ...     0     0     0]
 [14433 13513 15316 ...     0     0     0]] (19584, 100) 

[[   19     2   655 ...     0     0     0]
 [   37  1271   342 ...     0     0     0]
 [  303     1     1 ...     0     0     0]
 ...
 [  108    60  1113 ...     0     0     0]
 [  448   501    15 ...     0     0     0]
 [14433     0 13513 ...     0     0     0]] (19584, 100)
[0.24430147 0.75569853]


## Define LSTM Model

The codeblock below implements a Bidirectional LSTM model. The design of this is adapted from https://medium.com/southpigalle/simple-relation-extraction-with-a-bi-lstm-model-part-1-682b670d5e11.

In [ ]:
class RelExLSTM(Model):
    def __init__(self, input_size, embedding_size, lstm_units, vocab_size, num_labels, max_len, vocab, use_vocab=False):
        super(RelExLSTM, self).__init__()

        if use_vocab:
            self.vectoriser = TextVectorization(vocabulary=vocab)
        else:
            self.vectoriser = TextVectorization(max_tokens=vocab_size, output_sequence_length=max_len)
            self.vectoriser.adapt(vocab)

        self.embedding = Embedding(vocab_size, embedding_size, input_length=input_size)
        self.lstm = Bidirectional(LSTM(lstm_units, dropout=0.7, recurrent_dropout=0.7))
        self.dense = Dense(num_labels, activation='softmax')

    def call(self, inputs):
        x = self.vectoriser(inputs)
        x = self.embedding(x)
        x = self.lstm(x)
        y = self.dense(x)
        return y





model = RelExLSTM(INPUT_SIZE, EMEDDING_SIZE, LSTM_UNITS, VOCAB_SIZE, NUM_LABELS)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["acc"])